# IaC: Infrastructure-as-code

### This notebook handles the following tasks:<br>
- Load all the DWH parameters from the configuration file <br>
- Create clients for AWS services: S3, EC2, IAM and Redshift <br>
- Create an IAM role to allow Redshift cluster to access S3 buckets <br>
- Create a Redshift cluster <br>
- Open an incoming TCP port to access the cluster ednpoint <br>
- Connect to the cluster <br>
- Delete the created cluster <br>
- Delete all the created resources <br>

In [ ]:
# import libraries
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import json
import configparser
%load_ext sql

## Load DWH params from the configuration file (dwh.cfg)

In [ ]:
# initilise a configparser object

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# load configurated values into local variables

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE = config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUMBER_NODES = config.get('DWH','DWH_NUMBER_NODES')
DWH_NODE_TYPE = config.get('DWH','DWH_NODE_TYPE')
DWH_IAM_ROLE_NAME = config.get('DWH','DWH_IAM_ROLE_NAME')
DWH_CLUSTER_IDENTIFIER = config.get('DWH','DWH_CLUSTER_IDENTIFIER')

DB_NAME = config.get('CLUSTER','DB_NAME')
DB_USER = config.get('CLUSTER','DB_USER')
DB_PASSWORD = config.get('CLUSTER','DB_PASSWORD')
DB_PORT = config.get('CLUSTER','DB_PORT')

LOG_DATA = config.get('S3','LOG_DATA')
LOG_JSONPATH = config.get('S3','LOG_JSONPATH')
SONG_DATA = config.get('S3','SONG_DATA')

# view the values as a dataframe
pd.DataFrame({'Param' : ['DWH_CLUSTER_TYPE', 'DWH_NUMBER_NODES', 'DWH_NODE_TYPE', 'DWH_CLUSTER_IDENTIFIER', 'DWH_IAM_ROLE_NAME', 'DB_NAME', 'DB_USER',
                         'DB_PASSWORD', 'DB_PORT', 'LOG_DATA', 'LOG_JSONPATH', 'SONG_DATA'],
             'Value' : [DWH_CLUSTER_TYPE, DWH_NUMBER_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_IAM_ROLE_NAME, DB_NAME, DB_USER, DB_PASSWORD,
                       DB_PORT, LOG_DATA, LOG_JSONPATH, SONG_DATA]})


## Create clients for S3, EC2, IAM and Redshift

In [ ]:
# initilize clients for amazon services using boto3

iam = boto3.client('iam',
                    region_name = 'us-west-2',
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

redshift = boto3.client('redshift',
                       region_name = 'us-west-2',
                       aws_access_key_id = KEY,
                       aws_secret_access_key = SECRET)

s3 = boto3.resource('s3',
                 region_name = 'us-west-2',
                 aws_access_key_id = KEY,
                 aws_secret_access_key = SECRET)

ec2 = boto3.resource('ec2',
                    region_name = 'us-west-2',
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

# Step1: Create an IAM role that makes Redshift able to access S3 bucket
- Copy the RoleARN to dwh.cfg

In [ ]:
# create an IAM role

try:
    print('1.1 Creating an IAM role')
    dwhRole = iam.create_role(
                Path = '/',
                RoleName = DWH_IAM_ROLE_NAME,
                Description = 'Allow Redshift cluster to call AWS services on your behalf.',
                AssumeRolePolicyDocument = json.dumps({'Statement': [{'Action': 'sts:AssumeRole',
                                           'Effect': 'Allow',
                                           'Principal': {'Service': 'redshift.amazonaws.com'}}],
                                            'Version': '2012-10-17'}))

except Exception as e:
    print(e)

    
# attach the role policy

print('1.2 Attaching policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

# get the IAM role ARN

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

# Step2: Create Redshift cluster

In [ ]:
# create the Redshift cluster

try:
    response = redshift.create_cluster(
                #HW
                ClusterType=DWH_CLUSTER_TYPE,
                NodeType=DWH_NODE_TYPE,
                NumberOfNodes=int(DWH_NUMBER_NODES),

                #Identifiers & Credentials
                DBName=DB_NAME,
                ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
                MasterUsername=DB_USER,
                MasterUserPassword=DB_PASSWORD,
        
                #Roles (for s3 access)
                IamRoles=[roleArn] ) 
    
except Exception as e:
    print(e)

## 2.1: Check the cluster status

In [ ]:
def prettyRedshiftProps(props):
    """ 
    A function to print the cluster properties as a dataframe.
    
    Parameters: 
    props (properties): cluster properties which are the results of a redshift.describe.
    """
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

## 2.2: Take a note of the cluster endpoint and role ARN
- Run this cell when the cluster status becomes avialable <br>
- Copy the DB_ENDPOINT to dwh.cfg

In [ ]:
# keep the DATABASE ENDPOINT and IAM ROLE ARN values 

DB_ENDPOINT = myClusterProps['Endpoint']['Address']
ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DB_ENDPOINT :: ", DB_ENDPOINT)
print("ROLE_ARN :: ", ROLE_ARN)

# Step3: Open an incoming TCP port to access the cluster ednpoint

In [ ]:
# open a TCP port to access the cluster endpoint

try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

# Step4: Connect to the cluster

In [ ]:
# connect to the cluster

conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DB_ENDPOINT, DB_PORT, DB_NAME)
print(conn_string)
%sql $conn_string

# Step5: Delete the created cluster and resources
Only run when you finish with the project

In [ ]:
# delete the created cluster
# uncomment to run 
#redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

In [ ]:
# print cluster properties to check it's statust

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# delete the created resources
# uncomment to run 

#iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
#iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)